## Do Phonon calculations for $\ce{LiFeOxF}$

In [1]:
%aiida

Loaded AiiDA DB environment - profile name: scanlon-test.

In [2]:
base_structure = load_node('373caa6e-ce4a-48e1-b543-de4050665691')
print(base_structure.label)

2LiFeOxF-210401-125112-a0dc43 RELAXED


In [3]:
from aiida_user_addons.process.transform import niggli_reduce, make_supercell

In [4]:
reduced = niggli_reduce(base_structure)

09/15/2021 03:15:21 PM <21423> aiida.orm.nodes.process.calculation.calcfunction.CalcFunctionNode: [INFO] Process<623927>: Broadcasting state change: state_changed.created.running
09/15/2021 03:15:22 PM <21423> aiida.orm.nodes.process.calculation.calcfunction.CalcFunctionNode: [INFO] Process<623927>: Broadcasting state change: state_changed.running.finished


In [5]:
reduced = load_node('4a7aa0a3-3e32-44f5-9a9c-cf1e2db0a5bc')

In [10]:
import numpy as np
from aiida_user_addons.common.inputset.vaspsets import VASPInputSet
import aiida.orm as orm
from aiida_user_addons.vworkflows.phonon_wc import PhononSettings
from aiida.engine import submit

def setup_auto_phonon(structure, supercell_matrix, ncore=6, do_relax=True, init_relax_force_tol=1e-4, ncpus_frame=48, ncpus_relax=48, ncpus_nac=48,
                      code_string='vasp-5.4.4-std-k@michael', ediff_super=1e-6, sp_extras=None, relax_extras=None, mesh=100, symmetry_tolerance=1e-5):
    """
    Setup the automatric phonon workchain to perform high precision relaxation + finite displacement phonon calculation
    """
    supercell_matrix = np.asarray(supercell_matrix)
    gga = 'ps'  # Use PBESOL
    if sp_extras is None:
        sp_extras = {}
    if relax_extras is None:
        relax_extras = {}
        
    relax_set = VASPInputSet("UCLPhononSet", structure, {'gga': gga,  'ncore': ncore, 'ediff': 1e-8, 'kpar': 2, 'ispin': 2, 'potim': 0.3, 'kpar': 2, **relax_extras})
    sp_set = VASPInputSet("UCLPhononSet", orm.StructureData(pymatgen=structure.get_pymatgen() * supercell_matrix), {'gga': gga, 'ediff': ediff_super, 
                                                                                                                    'ncore': ncore, 'kpar': 2, 'ispin': 2, 'isym': 0, **sp_extras})
    nac_set = VASPInputSet("UCLPhononSet", structure, {'gga': gga,  'ediff': 1e-8, 'ncore': None, 'lepsilon': True, 'ispin': 2, 'isym': 0})

    VaspPhonon = WorkflowFactory('vaspu.phonopy')
    pp_family = Str("PBE.54")
    pp_mapping = Dict(dict=relax_set.get_pp_mapping())

    builder = VaspPhonon.get_builder()
    relax = builder.relax
    sp = builder.singlepoint
    nac = builder.nac

    ### Relaxation settings
    if do_relax:
        from aiida_user_addons.vworkflows.relax import RelaxOptions

        relax.relax_settings = RelaxOptions(force_cutoff=init_relax_force_tol, convergence_on=True, algo='rd').to_aiida_dict()
        relax.vasp.parameters = Dict(dict={'incar': relax_set.get_input_dict()})
        relax.vasp.options = Dict(dict={
            'resources': {'parallel_env': 'mpi', 'tot_num_mpiprocs': ncpus_relax},
            'max_wallclock_seconds': int(3600 * 24)
        })
        relax.vasp.potential_family = pp_family
        relax.vasp.potential_mapping = pp_mapping
        relax.vasp.kpoints_spacing = Float(0.04)
        relax.vasp.code = Code.get_from_string(code_string)
        # Fix problem with increased symprec
        relax.static_calc_parameters = Dict(dict={'incar': {'symprec': 1e-8}})
        
    ## Single point settings
    sp.parameters = Dict(dict={'incar': sp_set.get_input_dict()})
    sp.options = Dict(dict={
        'resources': {
            'parallel_env': 'mpi', 
            'tot_num_mpiprocs': ncpus_frame
        },
        'max_wallclock_seconds': int(3600 * 24)
    })
    sp.potential_family = pp_family
    sp.potential_mapping = pp_mapping
    sp.kpoints_spacing = Float(0.04)
    sp.code = Code.get_from_string(code_string)

    ## NAC settings
    nac.parameters = Dict(dict={'incar': nac_set.get_input_dict()})
    nac.options = Dict(dict={
        'resources': {'parallel_env': 'mpi', 'tot_num_mpiprocs': ncpus_nac},
        'max_wallclock_seconds': int(3600 * 12)
    })
    nac.potential_family = pp_family
    nac.potential_mapping = pp_mapping
    nac.kpoints = nac_set.get_kpoints(0.04)
    nac.code = Code.get_from_string(code_string)

    ## Phonon settings
    settings = PhononSettings(supercell_matrix=supercell_matrix.tolist(), 
                              mesh=mesh)
    settings.primitive_matrix = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    builder.phonon_settings = settings.to_aiida_dict()
    builder.symmetry_tolerance = Float(symmetry_tolerance)
    # Run supercell calculation and reuse the CHGCAR
    builder.reuse_supercell_calc = Str("restart")


    builder.structure = structure
    if supercell_matrix.ndim == 1:
        builder.metadata.label = structure.label + " SUPER {} {} {}".format(*supercell_matrix) # INPUT the label here
    else:
        builder.metadata.label = structure.label + " SUPER {:.0f}".format(np.linalg.det(supercell_matrix)) # INPUT the label here

    return builder

In [11]:
from ase.build import make_supercell, find_optimal_cell_shape
from spglib import standardize_cell
from ase import Atoms

In [12]:
transform_mat = np.array([
    [2, 0, 0],
    [-1, 2, 0],
    [0, -1, 2],
])
sout = make_supercell(reduced.get_ase(), transform_mat)

In [13]:
supercell = load_node('9fd75b0a-aa8c-47d8-9ebb-9b6a904d4d85')

Note - transpose of the transform matrix is needed